In [ ]:
import os
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras import layers, models, optimizers
from tensorflow.keras.applications.vgg16 import VGG16
from tensorflow.keras.layers import Flatten, Dense, Dropout

# ---------- USER SETTINGS ----------
DATASET_DIR = "Downloads/LP-IV-datasets-20251107T052642Z-1-001/LP-IV-datasets/Object Detection(Ass6)/caltech-101-img"
VGG_WEIGHTS_PATH = "Downloads/LP-IV-datasets-20251107T052642Z-1-001/LP-IV-datasets/Object Detection(Ass6)/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"
IMG_SIZE = (224, 224)
BATCH_SIZE = 32
SEED = 1337

# 1) Load dataset (labels will be one-hot)
train_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="training",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'   # one-hot labels (works with categorical_crossentropy)
)
val_data = tf.keras.preprocessing.image_dataset_from_directory(
    DATASET_DIR,
    validation_split=0.2,
    subset="validation",
    seed=SEED,
    image_size=IMG_SIZE,
    batch_size=BATCH_SIZE,
    label_mode='categorical'
)

class_names = train_data.class_names
num_classes = len(class_names)
print(f"Found {num_classes} classes. Example: {class_names[:8]}")

# Speed: cache + prefetch (optional)
AUTOTUNE = tf.data.AUTOTUNE
train_data = train_data.cache().prefetch(buffer_size=AUTOTUNE)
val_data = val_data.cache().prefetch(buffer_size=AUTOTUNE)

# 2) Load local VGG16 without top (use only local weights)
base_model = VGG16(weights=VGG_WEIGHTS_PATH, include_top=False, input_shape=(*IMG_SIZE, 3))

# Freeze base model
base_model.trainable = False

# 3) Build simple classifier on top
model = models.Sequential([
    base_model,
    Flatten(),                       # simple and easy to understand
    Dense(512, activation='relu'),
    Dropout(0.5),
    Dense(256, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

model.summary()

# 4) Compile
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 5) Train (initial)
history = model.fit(
    train_data,
    validation_data=val_data,
    epochs=8
)

# 6) Optional: fine-tune last few layers of VGG (uncomment to use)
# for layer in base_model.layers[-4:]:
#     layer.trainable = True
# model.compile(optimizer=optimizers.Adam(1e-5), loss='categorical_crossentropy', metrics=['accuracy'])
# history_ft = model.fit(train_data, validation_data=val_data, epochs=4)

# 7) Evaluate
loss, acc = model.evaluate(val_data)
print(f"Validation Accuracy: {acc*100:.2f}%")

# 8) Plot (training only)
plt.plot(history.history['accuracy'], label='Train Acc')
plt.plot(history.history['val_accuracy'], label='Val Acc')
plt.title("Accuracy")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()


Found 9145 files belonging to 102 classes.
Using 7316 files for training.
Found 9145 files belonging to 102 classes.
Using 1829 files for validation.
Found 102 classes. Example: ['BACKGROUND_Google', 'Faces', 'Faces_easy', 'Leopards', 'Motorbikes', 'accordion', 'airplanes', 'anchor']


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ vgg16 (Functional)              │ (None, 7, 7, 512)      │    14,714,688 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_1 (Flatten)             │ (None, 25088)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 512)            │    12,845,568 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 512)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 256)            │       131,328 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_3 (Dropout)             │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 102)            │        26,214 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 27,717,798 (105.74 MB)

 Trainable params: 13,003,110 (49.60 MB)

 Non-trainable params: 14,714,688 (56.13 MB)

Epoch 1/8
 80/229 ━━━━━━━━━━━━━━━━━━━━ 9:26 4s/step - accuracy: 0.1847 - loss: 31.5762 